# Loan Approal Prediction


#### Problem Statement

>- Banks and financial institutions face the challenge of accurately assessing the risk of a borrower approval on a loan, which can result in a loss of revenue and damage to their reputation. 

>- Identifying potential fraud or misrepresentation by borrowers is also a significant concern for these institutions.

>- Therefore, there is a need to develop a loan  approval prediction model using Weka that can analyze a borrower's data and provide accurate predictions on the likelihood of being granted a loan.

### Import Libraries